In [2]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [3]:
%cd ../gdrive/MyDrive/bigdata


/gdrive/MyDrive/bigdata


In [4]:
!ls


amazon-fine-food-reviews.zip  hashes.txt   Reviews.csv	X.pkl
database.sqlite		      kaggle.json  reviews.txt


In [ ]:
!pip install pickle-mixin --user

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import pandas as pd
import nltk
import re
import string
import random 
import numpy as np
import pickle
import scipy

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [ ]:
!chmod 600 /content/kaggle.json

In [ ]:
!kaggle datasets download -d snap/amazon-fine-food-reviews

 98% 238M/242M [00:01<00:00, 193MB/s]
100% 242M/242M [00:01<00:00, 179MB/s]


In [ ]:
!ls

amazon-fine-food-reviews.zip  kaggle.json  reviews.txt	X.pkl


In [ ]:
from zipfile import ZipFile

In [ ]:
# Create a ZipFile Object and load sample.zip in it
with ZipFile('amazon-fine-food-reviews.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

In [ ]:
#import nltk

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

from nltk.tokenize import word_tokenize #Used to extract words from documents
from nltk.stem import WordNetLemmatizer #Used to lemmatize words

#import tfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
!ls

amazon-fine-food-reviews.zip  hashes.txt   Reviews.csv	X.pkl
database.sqlite		      kaggle.json  reviews.txt


In [ ]:

#read data from csv and get the reviews
df = pd.read_csv('Reviews.csv')
reviews=df["Text"].tolist()
#shuffle to later divide in test and training without bias
random.shuffle(reviews)

In [ ]:
#preprocess the reviews
contractions_dict = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "i'd": "I would",
  "i'd've": "I would have",
  "i'll": "I will",
  "i'll've": "I will have",
  "i'm": "I am",
  "i've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you will",
  "you'll've": "you will have",
  "you're": "you are",
  "you've": "you have"
}

# Regular expression for finding contractions
def multiple_replace(dict, text):
  # Create a regular expression  from the dictionary keys
  regex = re.compile("(%s)" % "|".join(map(re.escape, dict.keys())))

  # For each match, look-up corresponding value in dictionary
  return regex.sub(lambda mo: dict[mo.string[mo.start():mo.end()]], text)

reviews = [multiple_replace(contractions_dict, doc) for doc in reviews]
table = str.maketrans('', '', string.punctuation)
reviews = [doc.translate(table) for doc in reviews]
reviews = [re.sub(r'\d+', " ", doc) for doc in reviews]

In [ ]:
def get_lemmatized(doc):
  word_list = word_tokenize(doc)
  lemmatized_doc = ""
  for word in word_list:
    lemmatized_doc = lemmatized_doc + " " + lemmatizer.lemmatize(word)
  return lemmatized_doc

lemmatizer = WordNetLemmatizer()

reviews = [get_lemmatized(doc) for doc in reviews]

In [ ]:
stopwords = nltk.corpus.stopwords.words('english') # Returns a list
stopwords = set(stopwords) # We want a set, because this is implemented with a hash table
                           # Checking the if condition in rem_stop costs O(1) in this way

def rem_stop(doc):
   word_list = word_tokenize(doc)
   cleaned_doc = ""
   for word in word_list:
     if word not in stopwords:
       cleaned_doc += " " + word
   return cleaned_doc


reviews = [rem_stop(doc) for doc in reviews]

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english', min_df=10) ## Corpus is in English
X = vectorizer.fit_transform(reviews)

In [ ]:
# #write reviews to file
with open('reviews.txt', 'w',encoding='utf8') as f:
    for item in reviews:
        f.write(item+",")

In [ ]:
import pickle
#write x to file
with open('X.pkl', 'wb') as f:
	pickle.dump(X, f)

# read obj from file
with open('X.pkl', 'rb') as f:
	obj = pickle.load(f)

In [6]:
import pickle
#read reviews from file
with open('reviews.txt', 'r',encoding='utf8') as f:
    reviews = f.read().split(",")
    reviews = reviews[:-1]

#read X from file
with open('X.pkl', 'rb') as f:
    X = pickle.load(f)

rows,cols=X.shape

In [7]:
percentage=0.3
reviews=reviews[:int(len(reviews)*percentage)]
X=X[:int(X.shape[0]*percentage)]
X.shape



(170536, 33897)

In [8]:
class Operations: 

    def generateU():
        u=np.random.normal(0,1,cols)
        u=scipy.sparse.csr_matrix(u)
        return u

    def sparseProduct(x,y):
        return x.dot(y.transpose()).toarray()[0][0]

    def h(x,u):
        return np.sign(Operations.sparseProduct(x,u))

    def cosineSimilarity(x,y):
        r=Operations.sparseProduct(x,y)/(np.linalg.norm(x.toarray())*np.linalg.norm(y.toarray()))
        return r

    def jaccardSimilarity(x,y):
        intersection=len(x.intersection(y))
        union=len(x.union(y))
        return intersection/union





In [9]:
class Hash:

    def __init__(self,nBytes=50,inputDimension=10):
        self.nBytes=nBytes
        self.inputDimension=inputDimension
        self.reset()
    
    def reset(self):
        self.matrix=np.random.randn(self.nBytes,self.inputDimension)

    def generateHash(self,input):
        if type(input)==list:
            input=np.array(input)
        bools = (np.dot(input, self.matrix.T) > 0).astype('int')
        return ''.join(bools.astype('str'))
h=Hash()


In [13]:
h.generateHash(np.ones(10))

'01011000100010000100110001100111101110011111000100'

In [10]:

class Lsh:

    def __init__(self,X,reviews,b=10,r=10,testPercentage=0.005,nHashes=100):
        self.X=X
        self.reviews=reviews
        self.rows=X.shape[0]
        self.cols=X.shape[1]
        self.b=b
        self.r=r
        self.testPercentage=testPercentage
        self.nHashes=nHashes
        self.splitDataset()

    def setBR(self,b,r):
        self.b=b
        self.r=r
        
    def splitDataset(self):
        indexSplit=int(self.rows*self.testPercentage)
        self.testReviews,self.trainReviews=self.reviews[:indexSplit],self.reviews[indexSplit:]
        self.testX,self.trainX=self.X[:indexSplit],self.X[indexSplit:]
        self.rowsTest,self.colsTest=self.testX.shape
        self.rowsTrain,self.colsTrain=self.trainX.shape

    def generateM(self):
        self.mTest=[[]for i in range(self.rowsTest)]
        self.mTrain=[[]for i in range(self.rowsTrain)]
        for i in range(self.nHashes):
            u=Operations.generateU()
            for j in range(self.rowsTest):
                h=Operations.h(self.testX[j],u)
                self.mTest[j].append(h)
            for j in range(self.rowsTrain):
                h=Operations.h(self.trainX[j],u)
                self.mTrain[j].append(h)




    def lsh(self):
        self.dicts=[{} for i in range(self.b)]
        self.hashesUsed=[Hash(inputDimension=self.r) for i in range(self.b)]
        for i in range(self.b):
            for j in range(self.rowsTrain):
                v=self.mTrain[j][i*self.r:(i+1)*self.r]
                key=self.hashesUsed[i].generateHash(v)
                if key not in self.dicts[i]:
                    self.dicts[i][key]=set()
                self.dicts[i][key].add(j)



    def generateMMM(self):
      allU=np.random.normal(0,1,[self.cols,self.nHashes])
      allU=scipy.sparse.csr_matrix(allU)
      array=np.array((self.testX * allU).toarray())
      self.mTest=np.sign(array)
      array=np.array((self.trainX * allU).toarray())
      self.mTrain=np.sign(array)


    def lshMM(self):
      hashes=np.random.randn(self.b,30,self.r)
      self.hashesMM=hashes

      signatureTrain=self.mTrain[:,0:self.r*self.b].reshape(-1,self.b,self.r)
      self.signatureTrain=np.einsum("abr, bdr-> ab",signatureTrain,hashes)

      signatureTest=self.mTest[:,0:self.r*self.b].reshape(-1,self.b,self.r)
      self.signatureTest=np.einsum("abr, bdr-> ab",signatureTest,hashes)

  
      
    def findCosineSimilarsApproximatedMM(self):
        self.approximatedSimilars={}
        for i,_ in enumerate(self.signatureTest):
            for j in range(self.b):
              key=self.signatureTest[i][j]
              indices=np.nonzero(self.signatureTrain[:,j].T == key)
              # print(indices)
              if i not in self.approximatedSimilars:
                self.approximatedSimilars[i]=set()
              for k in indices[0].tolist():
                self.approximatedSimilars[i].add(k)
        return self.approximatedSimilars

    def findExactCosineSimilarsMM(self,threshold=0.5):
        self.exactSimilars={}

        sim=self.testX.dot(self.trainX.T)
        normTest=scipy.sparse.linalg.norm(self.testX,axis=1)
        normTrain=scipy.sparse.linalg.norm(self.trainX,axis=1)
        
  
        sim=sim/normTrain
        sim=(sim.T/normTest).T
        
        similars=(sim>=threshold).nonzero()
        for i in range(len(similars[0])):
          if similars[0][i] not in self.exactSimilars:
            self.exactSimilars[similars[0][i]]=set()
          self.exactSimilars[similars[0][i]].add(similars[1][i])
          
        return self.exactSimilars

        





        # self.exactSimilars={}
        # for i,v in enumerate(self.testX):
        #     for j,u in enumerate(self.trainX):
        #         r=Operations.cosineSimilarity(v,u)
        #         if r>=threshold:
        #             if i not in self.exactSimilars:
        #                 self.exactSimilars[i]=set()
        #             self.exactSimilars[i].add(j)
        # return self.exactSimilars

    def optimizeRBMM(self,cosineSimilarity=0.2,nHashes=10):
      self.findExactCosineSimilarsMM(cosineSimilarity)
      self.nHashes=nHashes
      maxTaken=0
      lsh.generateMMM()

      #automatic search for b and r
      maxJaccard=0
      maxR=0
      maxB=0
      for i in range(1,int(lsh.nHashes**0.5)):
        lsh.r=i
        lsh.b=int(lsh.nHashes/i)
        lsh.lshMM()
        b=lsh.findCosineSimilarsApproximatedMM()
        jacc=lsh.jaccardAB()
        if jacc>maxJaccard:
          maxJaccard=jacc
          maxR=lsh.r
          maxB=lsh.b
          maxTaken=Operations.jaccardSimilarity(self.A.intersection(self.B),self.A)
        lsh.b=i
        lsh.r=int(lsh.nHashes/i)
        lsh.lshMM()
        b=lsh.findCosineSimilarsApproximatedMM()
        jacc=lsh.jaccardAB()
        if jacc>maxJaccard:
          maxJaccard=jacc
          maxR=lsh.r
          maxB=lsh.b
          maxTaken=Operations.jaccardSimilarity(self.A.intersection(self.B),self.A)

      print([maxJaccard,maxR,maxB])
      print(maxTaken)
      self.r=maxR
      self.b=maxB
      return [maxJaccard,maxR,maxB]






    def findExactCosineSimilars(self,threshold=0.5):
        self.exactSimilars={}
        for i,v in enumerate(self.testX):
            for j,u in enumerate(self.trainX):
                r=Operations.cosineSimilarity(v,u)
                if r>=threshold:
                    if i not in self.exactSimilars:
                        self.exactSimilars[i]=set()
                    self.exactSimilars[i].add(j)
        return self.exactSimilars


    def findCosineSimilarsApproximated(self):
        self.approximatedSimilars={}
        for i,v in enumerate(self.mTest):
            for j,hash in enumerate(self.hashesUsed):
                key=hash.generateHash(v[j*self.r:(j+1)*self.r])
                if key in self.dicts[j]:
                    for k in self.dicts[j][key]:
                        if i not in self.approximatedSimilars:
                            self.approximatedSimilars[i]=set()
                        self.approximatedSimilars[i].add(k)
        return self.approximatedSimilars
    
    def jaccardAB(self):
        self.A=set()
        self.B=set()
        for i in self.exactSimilars:
            for j in self.exactSimilars[i]:
                self.A.add((i,j))
        for i in self.approximatedSimilars:
            for j in self.approximatedSimilars[i]:
                self.B.add((i,j))
        return Operations.jaccardSimilarity(self.A,self.B)

    def optimizeRB(self,cosineSimilarity=0.2,nHashes=10):
      self.findExactCosineSimilars(cosineSimilarity)
      self.nHashes=nHashes
      maxTaken=0
      lsh.generateM()

      #automatic search for b and r
      maxJaccard=0
      maxR=0
      maxB=0
      for i in range(1,int(lsh.nHashes**0.5)):
        lsh.r=i
        lsh.b=int(lsh.nHashes/i)
        lsh.lsh()
        b=lsh.findCosineSimilarsApproximated()
        jacc=lsh.jaccardAB()
        if jacc>maxJaccard:
          maxJaccard=jacc
          maxR=lsh.r
          maxB=lsh.b
          maxTaken=Operations.jaccardSimilarity(self.A.intersection(self.B),self.A)
        lsh.b=i
        lsh.r=int(lsh.nHashes/i)
        lsh.lsh()
        b=lsh.findCosineSimilarsApproximated()
        jacc=lsh.jaccardAB()
        if jacc>maxJaccard:
          maxJaccard=jacc
          maxR=lsh.r
          maxB=lsh.b
          maxTaken=Operations.jaccardSimilarity(self.A.intersection(self.B),self.A)

      print([maxJaccard,maxR,maxB])
      print(maxTaken)
      self.r=maxR
      self.b=maxB
      return [maxJaccard,maxR,maxB]


  



In [11]:
lsh=Lsh(X,reviews,nHashes=100,r=5,b=20)


In [57]:
print(lsh.testX.shape,lsh.trainX.shape)

(852, 33897) (169684, 33897)


In [64]:
fromsim=0.5
top=1-np.arccos(fromsim)/np.pi
top

0.6666666666666666

In [75]:
fromp=0.6
tosim=np.cos((1-fromp)*np.pi)
tosim

0.30901699437494745

In [ ]:
lsh.r=10
lsh.b=100
lsh.nHashes=lsh.r*lsh.b
lsh.findExactCosineSimilarsMM(0.3)
lsh.generateMMM()
lsh.lshMM()
lsh.findCosineSimilarsApproximatedMM()
"finished"

/usr/local/lib/python3.7/dist-packages/scipy/sparse/base.py:595: RuntimeWarning: invalid value encountered in true_divide
  return np.true_divide(self.todense(), other)


In [ ]:
print(lsh.jaccardAB())
print(Operations.jaccardSimilarity(lsh.A.intersection(lsh.B),lsh.A))

